In [ ]:
import numpy as np

func = lambda x: np.abs(np.cos(np.sin(x)) - 3/4)

# Составной метод Симпсона: f - функция, [a, b] - отрезок, n - количество разбиений
def simpson_method(f, a, b, n):
    h = (b - a) / n  # Шаг интегрирования
    
    # Значение интеграла: h/3 * (y_0 + y_n + 4*сумма_нечетных + 2*сумма_четных)
    
    # Крайние точки
    integral = f(a) + f(b)
    
    # Сумма для нечетных индексов (1, 3, 5...) - коэффициент 4
    sum_odd = 0
    for i in range(1, n, 2):
        sum_odd += f(a + i * h)
        
    # Сумма для четных индексов (2, 4, 6...) - коэффициент 2
    sum_even = 0
    for i in range(2, n, 2):
        sum_even += f(a + i * h)
        
    integral = (h / 3) * (integral + 4 * sum_odd + 2 * sum_even)
    return integral

# Уточнение интеграла процессом Эйткина

def aitken_process(I_h, I_2h, I_4h):
    # Формула: I_refind = I_h - (I_2h - I_h)^2/(I_4h - 2*I_2h + I_h)

    numerator = (I_2h - I_h) ** 2
    denominator = I_4h - 2 * I_2h + I_h

    return I_h - (numerator / denominator)

def main():
    a, b = 0.0, np.pi
    epsilon = 1e-4      # Требуемая точность


    # Начальное количество разбиений
    n = 2
    
    # Список истории вычислений
    history = []
    
    # Первое приближение
    current_val = simpson_method(func, a, b, n)
    history.append(current_val)
    
    print(f"{n:<15} | {current_val:<20.8f} | {'-':<20}")

    # Основной цикл удвоения шагов
    while True:
        n *= 2  # Удваиваем число интервалов
        prev_val = current_val
        current_val = simpson_method(func, a, b, n)
        history.append(current_val)
        
        # Оценка погрешности по правилу Рунге:
        # Для метода Симпсона порядок точности p=4
        runge_error = abs(current_val - prev_val) / 15.0

        # Условие выхода: погрешность меньше epsilon.
        # Также проверяем, что у нас есть хотя бы 3 значения для применения процесса Эйткена.
        if runge_error < epsilon and len(history) >= 3:
            print("-" * 75)
            print("Достигнута точность!")
            break
    
    # Уточнение по Эйткену
    I_4h = history[-3]  # I при N/4
    I_2h = history[-2]  # I при N/2
    I_h  = history[-1]  # I при N
    
    refined_aitken = aitken_process(I_h, I_2h, I_4h)
    
    print("\n--- Процесс Эйткена ---")
    print(f"Используем значения при N = {n//4}, {n//2}, {n}")
    print(f"Уточненное значение интеграла: {refined_aitken:.10f}")

if __name__ == "__main__":
    main()

---------------------------------------------------------------------------
N (разбиений)   | Значение I           | Погрешность (Runge) 
---------------------------------------------------------------------------
2               | 0.70098921           | -                   
4               | 0.26215338           | 2.92557218e-02      
8               | 0.46598577           | 1.35888258e-02      
16              | 0.46232955           | 2.43747866e-04      
32              | 0.46009308           | 9.32481335e-05      
---------------------------------------------------------------------------
Достигнута точность!

Итоговое значение (Симпсон, N=32): 0.4600930800

--- Процесс Эйткена ---
Используем значения при N = 8, 16, 32
Уточненное значение интеграла: 0.4600641958


In [10]:
import numpy as np
from scipy import integrate

def func(x):
    return np.abs(np.cos(np.sin(x)) - 3/4)

exact_value, _ = integrate.quad(func, 0, np.pi)

print(f"Точное значение интеграла: {exact_value:.15f}")

Точное значение интеграла: 0.460043630499834
